In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from azureml.core import Model
from azureml.core import Experiment
from azureml.core import Workspace

In [2]:
import azureml.core
print(f'AZURE ML SDK VERSION: {azureml.core.VERSION}')

AZURE ML SDK VERSION: 1.31.0


In [3]:
ws = Workspace.from_config()
print(ws)

Workspace.create(name='mlops_ws', subscription_id='6f507867-3280-4f79-8820-9939bef45222', resource_group='learn_mlops')


### Define Environnement

In [4]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

In [5]:
env = Environment(name='my_env')

In [6]:
for pip_package in ["numpy", "onnxruntime", "joblib", "azureml-core", "azureml-monitoring", "azureml-defaults", 
                    "scikit-learn==0.22.2", "inference-schema", "inference-schema[numpy-support]"]:
    env.python.conda_dependencies.add_pip_package(pip_package)
    
inference_config = InferenceConfig(entry_script='score.py', environment=env)

In [7]:
env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
       

### Deployment configuration

In [8]:
from azureml.core.webservice import AksWebservice

deploy_config = AksWebservice.deploy_configuration(collect_model_data=True)
deploy_config

### Deploy webservice

In [9]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [11]:
# name for cluster Kubernetes
ask_name = 'myask-cluster'
#ask_target = ComputeTarget(workspace=ws, name=ask_name)
#print('Found existing cluster Kubernetes....')
# provisioning configuration with default parameters
prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
ask_target = ComputeTarget.create(workspace = ws, name = ask_name, provisioning_configuration = prov_config)
ask_target.wait_for_completion(show_output = True)

Creating.....................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [13]:
model1 = Model(workspace=ws, name='Scaler')
model2 = Model(workspace=ws, name="Support-vector-Classifier")

service_name = "weather-ask-prediction"

In [19]:
service = Model.deploy(ws, service_name, models=[model1, model2], inference_config=inference_config, deployment_config=deploy_config, 
                       deployment_target=ask_target)
service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-20 12:29:33+00:00 Creating Container Registry if not exists.
2021-07-20 12:29:33+00:00 Registering the environment.
2021-07-20 12:29:35+00:00 Use the existing image.
2021-07-20 12:29:37+00:00 Creating resources in AKS..
2021-07-20 12:29:37+00:00 Submitting deployment to compute.
2021-07-20 12:29:38+00:00 Checking the status of deployment weather-ask-prediction..
2021-07-20 12:31:18+00:00 Checking the status of inference endpoint weather-ask-prediction.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [20]:
print(service.get_logs())

2021-07-20T12:31:14,918049257+00:00 - iot-server/run 
2021-07-20T12:31:14,918118959+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-07-20T12:31:14,918850181+00:00 - rsyslog/run 
2021-07-20T12:31:14,996384270+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-07-20T12:31:15,086281140+00:00 - iot-server/finish 1 0
2021-07-20T12:31:15,088482208+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 40
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-07-20 12:31:16,808 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-07-20 12:31:16,809 | root | INFO | Starting up request id generator
2021-07-20 12:31:16,809 | root | INFO | Starting up app in

## Test web service

In [21]:
import json

input_payload = json.dumps({
    'data':[34.927778, 0.24, 7.3899, 83, 16.1000, 1]
})
output = service.run(input_payload)

print(output)

{'result': [1]}
